# Trabajo Fin de Master

<font size="4">**Universidad Internacional de Valencia**</font>

<font size="3">Título: “Aprendizaje profundo aplicado a física de partículas: estimación de trazas de partículas detectadas por el telescopio de neutrinos en el fondo marino.”</font>

Máster en Inteligencia Artificial

Director TFM: Dr. Salva Ardid Ramírez

Alumno: Ing. Marco Antonio Ortiz Meneses

Curso Académico: 2018-2019

## Introducción

Este es el Notebook que utilizaré para el desarrollo del TFM, y en el se contienen los aspectos técnicos relevantes para el tratamiento de los datos motivo del TFM.

Como parte fundamental, seguiré la metodología CRISP-DM, la cual conlleva seis fases:
* 1. Entendimiento del Negocio (o del Problema en este caso)
* 2. Entendimiento de los Datos
* 3. Preparación de los Datos
* 4. Modelado
* 5. Evaluación
* 6. Despliegue-Implementación del modelo en producción

**Es importante mencionar que los modelos se deben mantener y monitorear, ya que tienden a cambiar con el tiempo y a dejar de ser precisos, sobre todo si las circunstancias originales o variables cambian drásticamente.**

## 1. Entendimiento del Problema

Para el punto 1 de la Metodología, leí los documentos proporcionados por mi Director de TFM y que son:
* A fast algorithm for muon track reconstruction and its application to the ANTARES neutrino telescope
* Nuclear Instruments and Methods in Physics Research A

Una vez entendido a grandes razgos, como es que funciona el telescopio de neutrinos ANTARES y algunos detalles técnicos de operación, procedo a comenzar a analizar los archivos de datos proporcionados.

Es importante para mí el mencionar que he leído varias veces los documentos ya que no es fácil entender de primera instancia todas las variables involucradas y sobre todo la física relativa a los neutrinos y otras partículas asociadas a estos.

## 2. Entendimiento de los datos

In [1]:
#########################################################################################################
## Comenzare por importar los datos a partir del archivo (fichero) proporcionado                       ##
## Previamente descomprimí el archivo de 23GB en un directorio contenido en la variable datafilespath  ##
## Se requiere descomprimir todos y cada uno de los archivos a su formato txt primero                  ##
## Se deben depositar en un solo directorio representado por la variable extractpath                   ##
#########################################################################################################

import os
import gzip

# Si se desea se pueden modificar las variables
datafilespath = "C:\\NetworkTransfers\\DATA_FILES_TFM"
filesarr = os.listdir(datafilespath)
extractpath = datafilespath+"\\TXTs"

print("Numero de archivos a tratar:", len(filesarr))
print("Directorio en donde se extraeran los archivos:\n", extractpath)

Numero de archivos a tratar: 2555
Directorio en donde se extraeran los archivos:
 C:\NetworkTransfers\DATA_FILES_TFM\TXTs


Referencias para el tratamiento con gzip

* https://docs.python.org/3/library/gzip.html

* https://pymotw.com/2/gzip/

* https://stackoverflow.com/questions/41270130/how-to-decompress-a-file-using-python-3-x-and-gzip

In [ ]:
#####################################################################
### ESTA CELDA SOLO DEBE EJECUTARSE UNA VEZ SI Y SOLO SI NO SE HA ###
###       EJECUTADO PREVIAMENTE, POR FAVOR TENER CUIDADO          ###
### EN GENERAL NO DEBERÍA EJECUTARSE SOLO SE PONE COMO REFERENCIA ###
#####################################################################

# El siguiente código, nos sirve para descomprimir uno por uno
# todos los archivos despues de haberlos descomprimido del original de 23 GB
# ya que en el de 23GB venian comprimidos en formato gzip y se requiere una
# doble "descompresión".

# Es importante mencionar que esto me resulto más recomendable hacerlo
# en un directorio local ya que son 2555 archivos
# Adicionalmente el tiempo que se requiere para esta descompresión, es
# significativo, de alrededor de 30 min y es intensivo sobre todo en CPU.

for i in range(len(filesarr)-1):
    archivo = datafilespath+"\\"+filesarr[i]
    print("Ruta y archivo a descomprimir:\n", archivo)
    # Con lo siguiente leemos el contenido del archivo a descomprimir
    inF = gzip.open(archivo, 'rt') #Extracción como texto no binario
    datos = inF.read()
    inF.close()
    # Quitamos la extensión gz y ponemos el nombre correcto al archivo
    nombre = filesarr[i]
    nombredesc = nombre[:-3]
    # Escribimos el contenido al archivo ya descomprimido
    salida = extractpath+"\\"+nombredesc
    output = open(salida, 'wt')
    output.write(datos)
    output.close()
    print("\nTerminado el archivo:", nombredesc)
    
## Se debe verificar en la ruta extractpath que estén todos los archivos

### Referencia
### http://xahlee.info/python/gzip.html

**NOTAS DESPUÉS DE LA DESCOMPRESIÓN DE LOS ARCHIVOS:**

* Noto que existen "pares de archivos", es decir siempre hay uno con denominación "anumu" y otro con "numu" y tienen el mismo número.
* Despues de la descompresión, encuentro que algunos de los archivos tienen longitud 0 y habrá que ver como tratarlos, pero primero hay que tratar de analizar solo un par típico para ver con que data debemos quedarnos para su posterior tratamiento como vector.

In [2]:
#########################################################################################
#### ANÁLISIS DEL PRIMER ARCHIVO TÍPICO, EL QUE TIENE LA CADENA "anumu" EN EL NOMBRE ####
#########################################################################################

# Nombre del archivo a analizar: MC_025800_anumu_CC_a_reco.i3.gz.txt

# Necesitamos los nombres de los archivos en orden alfabetico
txtfilesarr = os.listdir(extractpath)

# Debemos extraer los datos relevantes de cada archivo, empezaremos analizando solo uno de ellos
# aunque posteriormente podemos poner esto en un bucle, pero hasta que haya dado con un metodo
# correcto para obtener solo los datos relevantes y "formar" un dataset que se pueda trabajar

f = open(extractpath+"\\"+txtfilesarr[0], 'rt')
contenido = f.read()
f.close

## Para ir viendo como es el archivo:
print ("Nombre del archivo: ",txtfilesarr[0],"\n")
print ("El contenido es de tipo: ",type(contenido),"\n")
print ("El tamaño de este primer archivo es: ", len(contenido))
#No puedeo ver todo porque el Notebook tiene una limitante de líneas
print (contenido[0:60000])

Nombre del archivo:  MC_025800_anumu_CC_a_reco.i3.gz.txt 

El contenido es de tipo:  <class 'str'> 

El tamaño de este primer archivo es:  5378539

start_event  1
25800 	73557 	0 	1170788183 	2007-02-06 18:56:23.000,000,000,0 UTC
weights  91530000.0 3191000.0 1000000.0
nu 0.0143410017774 -0.297532036876 0.954604118313 46.589 29.911 65.468 18.6806 -14
muon -0.0718400384771 -0.325562174369 0.942787504951 46.589 29.911 65.468 15.9965 -13
aafit  -0.294267285203 -0.300496709702 0.907253268011 41.085998826 -1.75032472806 142.220087928 -5.86985595954 0.130539349905
bbfit_track  nan nan 0.953328222068 0.094063685087 0.0 123.008344424 0.943145803497
bbfit_bright  nan nan nan 25.9873552501 0.0 90.055533167 2.55840589243
gridfit  0.294603614353 -0.0957014248113 0.950815412001 37.4964384115 13.8026415822 96.8343674639 6.1916762944
hit    1  3   2  1  -9.782512  37.711367 -156.955892  0.233261  0.667525 -0.707106   124.401378   1.230586   51.116943
hit    2  3   2  2 -10.540966  37.058107 -156.9558

**NOTAS DE LO QUE OBSERVO DEL PRIMER ARCHIVO TIPICO (anumu):**

* Se divide por bloques de "Eventos" (start_event 1, start_event 2...).
* Cuenta en la cabecera con 9 líneas que contienen diversos datos númericos separados por espacios en blanco.
* A partir de la décima línea empiezan los "hit".

* Cada evento cuenta con la siguiente información (por línea):

|Línea | Comentario|
|----|---|
| 1| Número del evento, p.ej. start_event 2|
| 2| 4 columnas numéricas, cuyo primer numero esta relacionado al nombre del archivo, las demás no estoy seguro, adicional la feha y hora del evento|
| 3| Pesos, aunque no se a que pesos se refiere|
| 4| Datos del neutrino (nu), con 8 columnas numéricas|
| 5| Datos del muón (mu), con 8 columnas numéricas|
| 6| Datos aafit, con 8 columnas numéricas|
| 7| Datos bbfit_track, los 2 primeros nan y son 7 columnas numéricas|
| 8| Datos bbfit_bright, los 3 primeros nan y son 7 columnas numéricas|
| 9| Datos gridfit, con 7 columnas numéricas|
|10| A partir de aqui son líneas con datos de hit, cada una con 13 columnas numéricas|

* Casi al final de cada evento, existe una línea que pone "BBFit selected pulses:" y a continuación viene una selección de hits.
* Finalmente existe un cadena que marca el final del evento identificada como "end_event".

Así sucesivamente hasta acabar el archivo.

Referencias para alinear tablas en Celdas Markdown y del bug que hace que todo salga a la derecha:

https://stackoverflow.com/questions/21892570/ipython-notebook-align-table-to-the-left-of-cell

https://github.com/jupyter/notebook/issues/3919

In [3]:
#########################################################################################
#### ANÁLISIS DEL SEGUNDO ARCHIVO TÍPICO, EL QUE TIENE LA CADENA "numu" EN EL NOMBRE ####
#########################################################################################

# Nombre del archivo a analizar: MC_025800_numu_CC_a_reco.i3.gz.txt

# Este parecer ser el "par" del primer archivo con nombre MC_025800_anumu_CC_a_reco.i3.gz.txt

# Debemos extraer los datos relevantes de cada archivo, continuare analizando el que parece el par
# posteriormente podemos poner esto en un bucle, pero hasta que haya dado con un metodo
# correcto para obtener solo los datos relevantes y "formar" un dataset que se pueda trabajar

f = open(extractpath+"\\"+txtfilesarr[1], 'rt')
contenido = f.read()
f.close

## Para ir viendo como es el archivo:
print ("Nombre del archivo: ",txtfilesarr[1],"\n")
print ("El contenido es de tipo: ",type(contenido),"\n")
print ("El tamaño de este primer archivo es: ", len(contenido))
#No puedeo ver todo porque el Notebook tiene una limitante de líneas
print (contenido[0:60000])

Nombre del archivo:  MC_025800_numu_CC_a_reco.i3.gz.txt 

El contenido es de tipo:  <class 'str'> 

El tamaño de este primer archivo es:  4653240

start_event  1
25800 	73409 	1 	1170799062 	2007-02-06 21:57:42.000,000,000,0 UTC
weights  129500000.0 8988000.0 1000000.0
nu -0.524274155595 0.21180406286 0.824788244782 6.789 86.746 -83.317 16.1684 14
muon -0.390696002439 0.215883001348 0.894847005587 6.789 86.746 -83.317 10.5977 -13
aafit  -0.485966740605 0.190895362792 0.852874719692 -10.3648248153 88.8487763655 -55.030587903 -5.92363117486 0.0231818150764
bbfit_track  nan nan 0.85565167819 0.0783090855826 0.0 -59.3038217525 1.17330870972
bbfit_bright  nan nan nan 13.7828590093 0.0 -81.0044249195 1.4624260416
gridfit  -0.0120095244647 0.826154182529 0.563316108425 -10.5830435425 101.918902378 -72.9776980196 6.38125052125
hit    1  3   2  2 -10.540966  37.058107 -156.955892 -0.694724 -0.131753 -0.707106 -1525.895470  1.383367   55.351257
hit    2  4  17  0  44.562730   9.519969   60.98410

**NOTAS DE LO QUE OBSERVO DEL SEGUNDO ARCHIVO TIPICO (numu):**

* Al igual que el anterior, se divide en los mismos "bloques"
* El numero del archivo (segunda línea primera cifra), es el mismo
* Los datos numéricos en general son diferentes.

In [11]:
### ANALIZANDO OTRO PAR DE ARCHIVOS CUALQUIERA, PERO QUE SEAN "PAREJA" ###

f = open(extractpath+"\\"+txtfilesarr[74], 'rt')
contenido_anumu = f.read()
f.close

f = open(extractpath+"\\"+txtfilesarr[75], 'rt')
contenido_numu = f.read()
f.close

print ("Nombre del archivo 1: ",txtfilesarr[74])
print ("Nombre del archivo 2: ",txtfilesarr[75],"\n")

print ("El tamaño del archivo 1 es: ", len(contenido_anumu))
print ("El tamaño del archivo 2 es: ", len(contenido_numu))

print ("\nContenido significativo del primer archivo:")
print (contenido_anumu[0:1000],"\n",".\n",".\n",".\n")
print (contenido_anumu[-1000:])

print ("\nContenido del segundo archivo:")
print (contenido_numu[0:1000],"\n",".\n",".\n",".\n")
print (contenido_numu[-1000:])

Nombre del archivo 1:  MC_027290_anumu_CC_a_reco.i3.gz.txt
Nombre del archivo 2:  MC_027290_numu_CC_a_reco.i3.gz.txt 

El tamaño del archivo 1 es:  5325166
El tamaño del archivo 2 es:  5458056

Contenido significativo del primer archivo:

start_event  1
27290 	46948 	0 	1177180555 	2007-04-21 18:35:55.000,000,000,0 UTC
weights  195100000.0 2997000.0 1000000.0
nu 0.421041833361 -0.152586939609 0.894114646128 -10.887 102.524 -71.532 24.4564 -14
muon 0.453500107446 -0.191052045265 0.870538206253 -10.887 102.524 -71.532 18.9465 -13
aafit  0.536296374348 -0.0498749133175 0.842554859865 35.9026716499 93.4859241767 6.83945250015 -5.23930977983 0.0182452714425
bbfit_track  nan nan 0.87037072855 0.0195796812607 0.0 -42.9336181999 0.695717551688
bbfit_bright  nan nan nan 11.0764674086 0.0 -58.5484668846 1.14059145634
gridfit  0.346773784781 -0.414738056357 0.841273015612 9.79974571331 87.327116506 -26.8902470235 6.29987681745
hit    1  3   2  0  -9.595928  36.727814 -156.955892  0.461464 -0.5357

**NOTAS:**

* Se puede observar que en la segunda línea las columnas 1 y 4, contienen el mismo valor numérico, así como la fecha y hora.
* Se mantiene la misma estructura, pero con valores totalmente diferentes.
* Los nan de las líneas 7 y 8 (bbfit...), se mantienen igual 2 en la primera y 3 en la segunda.

Para entender mejor el problema, es muy importante hacer la relación de los datos que vienen en los archivos, con lo que menciona el documento llamado **"A fast algorithm for muon track reconstruction and its application to the ANTARES neutrino telescope"**

Una vez "entendida" la estructura de los archivos, paso a revisar las instrucciones proporcionadas por mi director de TFM, aquí es importante mencionar que seguimos en la fase 2 de la metodología CRISP-DM.

En las instrucciones se detalla que:

* "Para el neutrino (nu) y el muon tenemos en las diferentes columnas: dirección (X Y Z, coordenadas cartesianas del vector unitario), posición (X' Y' Z'), energia, tipo_de_partícula. 
* La información importante para el proyecto es la del muón porque es la que se pretende predecir.
* Se puede trabajar con azimut y zenit, o con la dirección (X Y Z). Quizás lo mejor sea pasar a dos puesto que el interés máximo está en la predicción de valores de azimut (y zenit). Esto se puede hacer mediante la transformación azimut = atan2(Y/X) valores entre -pi i pi, y zenit = acos(Z), valores entre 0 i pi.
* Como digo arriba, ésta es la información que queremos predecir en los datos reales (generalización). Por tanto los valores de azimut y zenit (transformados de X Y Z) establecidos mediante simulación, los usaremos para corregir errores en el algoritmo supervisado durante el entrenamiento+validación."

In [23]:
### Prueba para ver cuantos eventos hay en cada uno de los archivos
import pandas as pd

f = open(extractpath+"\\"+txtfilesarr[0], 'rt')
data = f.read()
f.close

df = pd.DataFrame([x.split(';') for x in data.split('\n')])

df.shape

(46854, 1)

**Una vez que comprenda bien cada "par de archivos", debo saber qué es lo que debo "eliminar" para posterior tratamiento**

In [ ]:
# Una vez logrado debo proceder al tratamiento inicial y 
# a comenzar a guardar en vectores (numpy o pandas) la información relevante